# Aggregate swaps

This notebook shows a method to get all Bancor trades over a time period.

Set the timestamps and sum the trade volume to get 24h volume, daily trading volume, etc.

This is not a suitable method over larger timeframes.


## Setup
---

In [1]:
from graphqlclient import GraphQLClient
import pandas as pd
import json
from pandas.io.json import json_normalize
from datetime import datetime

In [2]:
ENDPOINT = "https://api.thegraph.com/subgraphs/name/blocklytics/bancor"
client = GraphQLClient(ENDPOINT)

In [3]:
# Timestamps
start_time = int(datetime(2020,1,1).timestamp()) # Inclusive
end_time = int(datetime(2020,1,14).timestamp())   # Exclusive

## Fetch data
---

In [4]:
# Start with an empty df
df = pd.DataFrame()

# Results must be paginated
limit = 100
offset = 0
fetching_results = True

In [5]:
# Fetch paginated results
while fetching_results:
    QUERY = """
    {{
      swaps(
        first: {0}, 
        skip: {1},
        where:{{ 
          timestamp_gte:"{2}",
          timestamp_lt:"{3}"
        }},
        orderBy:timestamp,
        orderDirection:desc
      ) {{
        fromToken {{ id, symbol, decimals }}
        toToken {{ id, symbol, decimals }}
        amountPurchased
        amountReturned
        converterUsed {{ id }}
        timestamp
        transaction {{ id }}
      }}
    }}
    """.format(limit, offset, start_time, end_time)

    result = json.loads(client.execute(QUERY))
    _df = pd.DataFrame.from_records(json_normalize(result['data']['swaps']))
    
    # If df is empty, create it from new _df
    # Else append new _df to df
    if len(df.index) == 0:
        df = _df
    else:
        df = df.append(_df)
    
    # Prepare for pagination
    result_length = len(_df.index)
    if limit == result_length:
        offset += limit
    else:
        fetching_results = False

df.reset_index(drop=True, inplace=True)
df.sort_values('timestamp', ascending=False)
df.shape

(16894, 11)

# Format data
---
Convert wei values

In [6]:
df['amountPurchased_float'] = df.apply(lambda row: float(row['amountPurchased']) / 10 ** row['fromToken.decimals'], axis=1)
df['amountReturned_float'] = df.apply(lambda row: float(row['amountReturned']) / 10 ** row['toToken.decimals'], axis=1)

Get volume in base token terms (USDB or BNT)

In [7]:
df['volume_bnt'] = \
    df.apply(lambda row: row['amountPurchased_float'] if row['fromToken.id'] == "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c" else 0, axis=1)\
    + df.apply(lambda row: row['amountReturned_float'] if row['toToken.id'] == "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c" else 0, axis=1)

df['volume_usdb'] = \
    df.apply(lambda row: row['amountPurchased_float'] if row['fromToken.id'] == "0x309627af60f0926daa6041b8279484312f2bf060" else 0, axis=1)\
    + df.apply(lambda row: row['amountReturned_float'] if row['fromToken.id'] == "0x309627af60f0926daa6041b8279484312f2bf060" else 0, axis=1)\


Convert timestamp

In [8]:
df['timestamp'] = df.apply(lambda row: datetime.fromtimestamp(int(row['timestamp'])), axis=1)

Preview swaps

In [9]:
df[['transaction.id', 'timestamp', 'amountPurchased_float', 'fromToken.symbol', 'amountReturned_float', 'toToken.symbol', 'volume_bnt', 'volume_usdb']]\
    .head(5)


,transaction.id,timestamp,amountPurchased_float,fromToken.symbol,amountReturned_float,toToken.symbol,volume_bnt,volume_usdb
0,0x1bece952ffed4ccea833343484edc673408ae36d46cb...,2020-01-13 23:58:50,0.036040,ETH,23.856345,BNT,23.856345,0.000000
1,0x1bece952ffed4ccea833343484edc673408ae36d46cb...,2020-01-13 23:58:50,23.856345,BNT,5.118950,USDB,23.856345,0.000000
2,0x1bece952ffed4ccea833343484edc673408ae36d46cb...,2020-01-13 23:58:50,5.118950,USDB,112.120597,REN,0.000000,117.239547
3,0x7750603ac38b22c01b8ef93f3519a5f88e585acb36c4...,2020-01-13 23:54:15,14546.000000,FXC,212.786256,BNT,212.786256,0.000000
4,0x7750603ac38b22c01b8ef93f3519a5f88e585acb36c4...,2020-01-13 23:54:15,212.786256,BNT,0.320178,ETH,212.786256,0.000000


# Volume per day
---

Add date column

In [10]:
df['date'] = df['timestamp'].apply(lambda row: row.date())

## BNT

This does not include volume through USDB converters and excludes Smart Token -> Token trades.

In [11]:
df.groupby(by='date')[['volume_bnt']].sum()

,volume_bnt
date,
2020-01-01,3.168038e+06
2020-01-02,4.289466e+06
2020-01-03,9.264600e+05
2020-01-04,6.092817e+05
2020-01-05,6.741570e+05
2020-01-06,9.345999e+05
2020-01-07,9.274317e+05
2020-01-08,8.372953e+05
2020-01-09,7.526183e+05


# Volumes over period
---
## BNT Volume

In [12]:
df['volume_bnt'].sum()

15946826.561210565

## USDB Volume

Avoid double-counting volume by excluding BNT volume already counted above.

In [11]:
# df['volume_usdb'].sum() # This will double-count BNT volume from the BNT-USDB converter.

In [13]:
df[df['volume_bnt'] == 0]['volume_usdb'].sum() # This will not double-count BNT volume.

175647.63033848637

## Trades not counted

These trades were not counted as they did not include a base token of BNT or USDB.

In [14]:
df[(df['volume_bnt'] == 0) & (df['volume_usdb'] == 0)]

,amountPurchased,amountReturned,converterUsed.id,fromToken.decimals,fromToken.id,fromToken.symbol,timestamp,toToken.decimals,toToken.id,toToken.symbol,transaction.id,amountPurchased_float,amountReturned_float,volume_bnt,volume_usdb,date
56,493460000000000000,488059252765383157,0x3a8cc07f17eb10e628c74b1a442c7adc2bfd854d,18.0,0x0000000000085d4780b73119b644ae5ecd22b376,TUSD,2020-01-13 21:49:51,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0x5391c07fc37fc3201fcff7b48e33a8a5412d16ed0339...,0.493460,0.488059,0.0,0.0,2020-01-13
76,2704679117539962257408,19439979770031923597,0x66540a3fcd929774a8dab59d56fe7a2d3538450f,18.0,0x8a9c67fee641579deba04928c4bc45f66e26343a,JRT,2020-01-13 21:06:59,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0xc427f924d0c2e2cf21c3dec52db46ecc758ce2737e89...,2704.679118,19.439980,0.0,0.0,2020-01-13
86,1300000000000000000000,276510682924207719976,0x08b61ded2f558071fbdb827715e7aef16e76dd4f,18.0,0xbf2179859fc6d5bee9bf9158632dc51678a4100e,ELF,2020-01-13 20:43:26,18.0,0x0f2318565f1996cb1ed2f88e172135791bc1fcbf,ELFBNT,0xe04b8c5ecc8a66623ec0cac2f6761135b03370d9b0b8...,1300.000000,276.510683,0.0,0.0,2020-01-13
123,50718344031001255936,5940929369434917892,0x97cf22539646d5a264fb3fbb68bb0642d8ad2a66,18.0,0xb62132e35a6c13ee1ee0f84dc5d40bad8d815206,NEXO,2020-01-13 20:13:00,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0x947e4d79e570527bfa303b593b26492a798b51ac14de...,50.718344,5.940929,0.0,0.0,2020-01-13
182,6300000000000000000,6106749761404869277,0x06f7bf937dec0c413a2e0464bb300c4d464bb891,18.0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2020-01-13 19:32:36,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0xf8b27dbc0f6c134ef82c6fd437bfb295164f8c7ed206...,6.300000,6.106750,0.0,0.0,2020-01-13
184,672982338899999543762,18005189976716193924,0x66540a3fcd929774a8dab59d56fe7a2d3538450f,18.0,0x8a9c67fee641579deba04928c4bc45f66e26343a,JRT,2020-01-13 19:28:48,18.0,0x4827e558e642861cd7a1c8f011b2b4661f8d51fa,JRTUSDB,0x6be03f358922d60f9ab6572ee448c5c54f3c9dfa5b68...,672.982339,18.005190,0.0,0.0,2020-01-13
302,28080000000000000000,27302848125349247080,0x06f7bf937dec0c413a2e0464bb300c4d464bb891,18.0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2020-01-13 17:04:51,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0x67ef25e8853835aba52f1e88bc25e2c8ce2ad6a74be0...,28.080000,27.302848,0.0,0.0,2020-01-13
306,46000000000000000000,45026408809321182297,0x06f7bf937dec0c413a2e0464bb300c4d464bb891,18.0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2020-01-13 17:02:59,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0x34ac7c1d177f52d13516a7e122708f6136a0ae527216...,46.000000,45.026409,0.0,0.0,2020-01-13
319,1000000000000000000,983001660942812669,0x06f7bf937dec0c413a2e0464bb300c4d464bb891,18.0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2020-01-13 16:41:39,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0xea1d42aad9e463f6545ec51ba79bd72d29030a6dd94f...,1.000000,0.983002,0.0,0.0,2020-01-13
357,656710671740693459,656330365289341932,0x3a8cc07f17eb10e628c74b1a442c7adc2bfd854d,18.0,0x0000000000085d4780b73119b644ae5ecd22b376,TUSD,2020-01-13 15:09:01,18.0,0x309627af60f0926daa6041b8279484312f2bf060,USDB,0x5b8784d7ed45c09535d5ba2cdd2e1cdb2517edc58dc5...,0.656711,0.656330,0.0,0.0,2020-01-13


In [14]:
df.iloc[23]

amountPurchased                                      464000000000000000000
amountReturned                                     90930259354164736443066
converterUsed.id                0xaa8cec9cbd7d051ba86d9deff1ec0775bd4b13c5
fromToken.decimals                                                      18
fromToken.id                    0x0f9be347378a37ced33a13ae061175af07cc9868
fromToken.symbol                                                    AMNBNT
timestamp                                              2020-01-01 23:05:18
toToken.decimals                                                        18
toToken.id                      0x737f98ac8ca59f2c68ad658e3c3d8c8963e40a4c
toToken.symbol                                                         AMN
transaction.id           0xdde655ef629e07ca5adec27ef78865168e62195aac6b...
amountPurchased_float                                                  464
amountReturned_float                                               90930.3
volume_bnt               